# Cofactor swap targets

In [1]:
%run 01-model.ipynb

from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

'' is not a valid SBML 'SId'.


In [2]:
model.solver = "glpk" # what is this?

In [3]:
biomass = model.reactions.DRAIN_Biomass

In [4]:
biomass.lower_bound = 0.1 # OBS denne skal måske ikke være 0,1 tjek tutorial notebooken og forstå det


In [5]:
demand = model.reactions.DM_psi

The demand reaction for the "target" (psilocybin) is defined in "01-model.ipynb"

In [6]:
model.objective = demand

In [7]:
product_yield = product_yield(demand, model.reactions.BOUNDARY_GLCe)

In [12]:
CofactorSwapOptimization?

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

Our model has different syntax for the default cofactors, and therefore the parameter cofactor_id_swaps must be added to the CofactorSwapOptimization function input with the correct syntax. 

In [61]:
swap_opt = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']), candidate_reactions = ['r14'])

In [66]:
%%time
result = swap_opt.run(max_size=1)

Starting optimization at Sat, 28 Nov 2020 11:33:38


HBox()

Finished after 00:00:00
CPU times: user 36.9 ms, sys: 0 ns, total: 36.9 ms
Wall time: 35.2 ms


In [67]:
result

,index,targets,fitness


Cofactor swapping has no effect on the production flux of DM_psi. It could be that swapping all of the NADH for NADPH in the model was disadvantageous for some reactions in the cell that require NADH, and therefore next the pool of NADPH will be increased by only targeting the last reaction in Glycolysis, Glyceraldehyde 3-phosphate dehydrogenase, which produces NADH.  

In [77]:
model.reactions.r14

Reaction identifier,r14
Name,Glyceraldehyde 3-phosphate dehydrogenase
Memory address,0x07f907604def0
Stoichiometry,"NAD + PI + T3P1 <=> 13PDG + NADH NAD+ + Orthophosphate + D-Glyceraldehyde 3-phosphate <=> 1,3-Bisphospho-D-glycerate + NADH"
GPR,1145793
Lower bound,-1000.0
Upper bound,1000.0


In [79]:
swap_opt_r14 = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']), candidate_reactions = ['r14'])

In [80]:
%%time
result_r14 = swap_opt_r14.run(max_size=1)

Starting optimization at Sat, 28 Nov 2020 11:58:41


HBox()

Finished after 00:00:00
CPU times: user 1.86 s, sys: 12 ms, total: 1.88 s
Wall time: 1.87 s


In [81]:
result_r14

,index,targets,fitness


The cofactor swap targeted to the Glyceraldehyde 3-phosphate dehydrogenase reaction had no results, and therefore increasing the pool of available NADPH clearly has no effect on the DM_psi production flux. 